<a href="https://colab.research.google.com/github/Nightcrawler9x/Beyond_visible_spectrum/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Challenge

##Generalidades

In [31]:
#@title Imports
import gdown
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [32]:
#@title Funciones
def load_from_drive(file_id, name):
  gdown.download(id=file_id, output=name, quiet=False)
  return name

def zero_pad(x, target_shape):
    padded = np.zeros(target_shape, dtype=x.dtype)
    slices = tuple(slice(0, min(dim, target)) for dim, target in zip(x.shape, target_shape))
    padded[slices] = x[slices]
    return padded

In [33]:
#@title Carga de los datos
load_from_drive('1kVbztu0a5E3qefYhMNi7KUgzQXo12uOn', 'data.zip')
!unzip "/content/data.zip" -d "/content/data"

##Exploracion de los datos

In [34]:
df_train = pd.read_csv('/content/data/train.csv')
df_test = pd.read_csv('/content/data/test.csv')

In [35]:
df_train.head()

,id,label
0,sample697.npy,7
1,sample54.npy,81
2,sample2270.npy,4
3,sample1401.npy,99
4,sample1901.npy,43


In [36]:
df_test.head()

,id
0,sample1957.npy
1,sample1369.npy
2,sample782.npy
3,sample2255.npy
4,sample2160.npy


In [37]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2177 entries, 0 to 2176
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2177 non-null   object
 1   label   2177 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.1+ KB


In [38]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      545 non-null    object
dtypes: object(1)
memory usage: 4.4+ KB


In [39]:
print(len(df_train['label'].unique()))

101


In [40]:
dir_csv = '/content/data/train.csv'
dir_npys = '/content/data/ot/ot'

chunk_size = 32
EXPECTED_SHAPE = (128,128,125)

In [41]:
def chunk_generator(dir_csv, dir_npys, chunk_size=32, expected_shape=(128,128,125)):
    df = pd.read_csv(dir_csv)
    npys = []
    labels = []

    for idx, fila in df.iterrows():
        npy_path = os.path.join(dir_npys, fila['id'])

        try:
            npy = np.load(npy_path)

            if npy.shape != expected_shape:
                try:
                    npy = zero_pad(npy, expected_shape)
                except Exception as e:
                    print(f"Error haciendo padding: {npy.shape} en {e}")
                    continue

            npys.append(npy)
            labels.append(fila['label'])

            if len(npys) == chunk_size:
                X_chunk = np.array(npys)
                y_chunk = np.array(labels)
                yield X_chunk, y_chunk
                npys = []
                labels = []

        except Exception as e:
            print(f"Error cargando {npy_path}: {e}")

In [42]:

def extract_patches(X, y, patch_size=5):
    patches = []
    labels = []
    margin = patch_size // 2

    for i in range(X.shape[0]):
        cube = X[i]
        label = y[i]
        for row in range(margin, cube.shape[0] - margin):
            for col in range(margin, cube.shape[1] - margin):
                patch = cube[row-margin:row+margin+1, col-margin:col+margin+1, :]
                patches.append(patch)
                labels.append(label)

    patches = np.array(patches)
    labels = np.array(labels)
    return patches, labels

In [43]:
!pip install tensorflow

In [ ]:
### Imports
# === IMPORTS ===
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam


### Modelo Autoencoder
input_shape = (5, 5, 125)
flatten_dim = np.prod(input_shape)
latent_dim = 64

input_img = Input(shape=input_shape)
x = Flatten()(input_img)
encoded = Dense(256, activation='relu')(x)
encoded = Dense(latent_dim, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(encoded)
decoded = Dense(flatten_dim, activation='sigmoid')(decoded)
decoded_reshaped = Reshape(input_shape)(decoded)

autoencoder = Model(input_img, decoded_reshaped)
encoder = Model(input_img, encoded)
autoencoder.compile(optimizer=Adam(1e-3), loss='mse')

### Entrenamiento
all_encoded = []
all_labels = []

for epoch in range(3):
    print(f"\n🔁 Epoch {epoch+1}/10")
    for X_chunk, y_chunk in chunk_generator(dir_csv, dir_npys):
        X_patches, y_labels = extract_patches(X_chunk, y_chunk)
        X_patches = X_patches / np.max(X_patches)

        autoencoder.fit(X_patches, X_patches, epochs=1, verbose=0)

        encoded = encoder.predict(X_patches)
        all_encoded.append(encoded)
        all_labels.extend(y_labels)

X_encoded = np.vstack(all_encoded)
patch_labels = np.array(all_labels)

# Clasificador
le = LabelEncoder()
y_encoded = le.fit_transform(patch_labels)
y_onehot = to_categorical(y_encoded)

input_latent = Input(shape=(latent_dim,))
x = Dense(64, activation='relu')(input_latent)
x = Dense(32, activation='relu')(x)
output = Dense(y_onehot.shape[1], activation='softmax')(x)

classifier = Model(input_latent, output)
classifier.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

classifier.fit(X_encoded, y_onehot, epochs=10, batch_size=64, shuffle=True, validation_split=0.2)



🔁 Epoch 1/10
